In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d ismailnasri20/driver-drowsiness-dataset-ddd

Dataset URL: https://www.kaggle.com/datasets/ismailnasri20/driver-drowsiness-dataset-ddd
License(s): unknown
 99% 2.56G/2.58G [00:23<00:00, 223MB/s]
100% 2.58G/2.58G [00:23<00:00, 117MB/s]


In [ ]:
import zipfile
zip_ref=zipfile.ZipFile('/content/driver-drowsiness-dataset-ddd.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
!pip install mediapipe

ERROR: Could not find a version that satisfies the requirement splitfolders (from versions: none)
ERROR: No matching distribution found for splitfolders
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires 

In [ ]:
pip install split-folders

In [ ]:
import cv2
import mediapipe as mp
import os
from tqdm import tqdm

input_base = 'Driver Drowsiness Dataset (DDD)'
output_base = 'cropped_dataset'

os.makedirs(output_base, exist_ok=True)

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

all_images = []
for label in ['Drowsy', 'Non Drowsy']:
    input_path = os.path.join(input_base, label)
    for img_name in os.listdir(input_path):
        all_images.append((label, img_name))


for label, img_name in tqdm(all_images, desc="Processing Images", unit="img"):
    input_path = os.path.join(input_base, label, img_name)
    output_path = os.path.join(output_base, label)
    os.makedirs(output_path, exist_ok=True)

    image = cv2.imread(input_path)
    if image is None:
        continue

    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = image.shape
            xmin = int(bboxC.xmin * w)
            ymin = int(bboxC.ymin * h)
            box_width = int(bboxC.width * w)
            box_height = int(bboxC.height * h)

            cropped_face = image[max(0, ymin):ymin + box_height, max(0, xmin):xmin + box_width]

            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, cropped_face)
            break
    else:
        print(f"[Warning] No face detected in {input_path}")


In [ ]:
import os
import splitfolders

input_dir = "/content/Driver Drowsiness Dataset (DDD)"
output_dir = '/content/split_data'

os.makedirs(output_dir, exist_ok=True)

input_dir_extracted='/content/Driver Drowsiness Dataset (DDD)'

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, seed=1337, ratio=(.7, .3), move=False)

Copying files: 41793 files [00:21, 1945.78 files/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout,BatchNormalization,Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
early_stopping=EarlyStopping(monitor='val_accuracy',patience=3,restore_best_weights=True)

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/split_data/train',
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(150,150)
)
test_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/split_data/val',
    labels='inferred',
    label_mode='int',
    batch_size=64,
    image_size=(150,150)
)

Found 29254 files belonging to 2 classes.
Found 12539 files belonging to 2 classes.


In [ ]:

def process(image, label):
    image = tf.image.resize(image, [150, 150])
    image = tf.reduce_mean(image, axis=-1, keepdims=True)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


train_ds=train_ds.map(process)
test_ds=test_ds.map(process)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='same',activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3,3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(train_ds,epochs=10,validation_data=test_ds,callbacks=early_stopping)

In [ ]:
from google.colab import files
model.save("model_trained_on_DDD_16.h5")
files.download("model_trained_on_DDD_16.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

dataset_path = 'cropped_dataset'

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        num_images = len([
            f for f in os.listdir(class_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        print(f"{class_name}: {num_images} images")


Drowsy: 14528 images
Non Drowsy: 12133 images


In [ ]:
!kaggle datasets download -d muhammadhananasghar/oace-open-and-close-eyes-dataset

Dataset URL: https://www.kaggle.com/datasets/muhammadhananasghar/oace-open-and-close-eyes-dataset
License(s): CC0-1.0
 98% 506M/518M [00:01<00:00, 307MB/s]
100% 518M/518M [00:01<00:00, 420MB/s]


In [ ]:
zip_ref = zipfile.ZipFile("/content/oace-open-and-close-eyes-dataset.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
!kaggle datasets download -d akshitmadan/eyes-open-or-closed

Dataset URL: https://www.kaggle.com/datasets/akshitmadan/eyes-open-or-closed
License(s): unknown
  0% 0.00/22.6M [00:00<?, ?B/s]
100% 22.6M/22.6M [00:00<00:00, 1.20GB/s]


In [ ]:
import os
import splitfolders

input_dir = "/content/OACE"
output_dir = '/content/new_eye_data'

os.makedirs(output_dir, exist_ok=True)

input_dir_extracted='/content/OACE'

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, seed=1337, ratio=(.7, .3), move=False)

Copying files: 97860 files [00:12, 7581.09 files/s]


In [ ]:
train_ds3=tf.keras.utils.image_dataset_from_directory(
    directory='/content/new_eye_data/train',
    labels='inferred',
    label_mode='int',
    batch_size=128,
    image_size=(150,150)
)
test_ds3=tf.keras.utils.image_dataset_from_directory(
    directory='/content/new_eye_data/val',
    labels='inferred',
    label_mode='int',
    batch_size=128,
    image_size=(150,150)
)


Found 68501 files belonging to 2 classes.
Found 29359 files belonging to 2 classes.


In [ ]:
def process(image, label):
    image = tf.image.resize(image, [150, 150])
    image = tf.reduce_mean(image, axis=-1, keepdims=True)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label
train_ds3=train_ds3.map(process)
test_ds3=test_ds3.map(process)

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,1)))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(train_ds3,epochs=10,validation_data=test_ds3)

Epoch 1/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 89s 147ms/step - accuracy: 0.8311 - loss: 0.3504 - val_accuracy: 0.9758 - val_loss: 0.0682
Epoch 2/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 124s 129ms/step - accuracy: 0.9740 - loss: 0.0780 - val_accuracy: 0.9850 - val_loss: 0.0412
Epoch 3/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 71s 132ms/step - accuracy: 0.9840 - loss: 0.0489 - val_accuracy: 0.9880 - val_loss: 0.0347
Epoch 4/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 71s 133ms/step - accuracy: 0.9864 - loss: 0.0391 - val_accuracy: 0.9900 - val_loss: 0.0295
Epoch 5/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 71s 131ms/step - accuracy: 0.9890 - loss: 0.0329 - val_accuracy: 0.9886 - val_loss: 0.0361
Epoch 6/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 82s 133ms/step - accuracy: 0.9906 - loss: 0.0281 - val_accuracy: 0.9908 - val_loss: 0.0315
Epoch 7/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 81s 131ms/step - accuracy: 0.9929 - loss: 0.0223 - val_accuracy: 0.9926 - val_loss: 0.0243
Epoch 8/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 83s 133ms/step - accuracy: 0.9921 - loss: 

In [ ]:
model.save('model_trained_on_OACE.h5')
from google.colab import files
files.download('model_trained_on_OACE.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>